# part 6. 모델 저장하고 불러오기
---
모델을 저장하고 불러오는 것은 매우 중요한 문제이다. 신경망을 학습시킬때 원큐에 학습이 될거란 보장이 없지 않은가?

그럼 어떻게 저장할까?

In [ ]:
import torch
import torchvision.models as models

### 1. 모델 가중치 저장하고 불러오기
---
> PyTorch 모델은 학습한 매개변수를 state_dict라고 불리는 내부 상태 사전(internal state dictionary)에 저장합니다.  
> 이 상태 값들은 torch.save 메소드를 사용하여 저장(persist)할 수 있습니다:

1. 내부 상태 사전(state_dict)가 뭔데?

    - 그냥 딕셔너리이다. 

    - 단지 각 계층을 매개변수 Tensor로 매핑해서 학습 가능한 매개변수를 갖는 계층이 모델의 state_dict에 항목을 가지게 된다.

일단 모델의 가중치를 저장하려면 3가지 함수를 알아야 한다

1. torch.save
    - 객체를 디스크에 저장합니다. pickle 모듈을 이용하여 객체를 직렬화 하며, 이 함수를 사용하여 모든 종류의 모델, Tensor 등을 저장할 수 있습니다.

    - 형태 : torch.save(obj, f, pickle_module=pickle, pickle_protocol=DEFAULT_PROTOCOL, _use_new_zipfile_serialization=True)

        - 매개변수
            - obj : 저장할 객체
            - f : 파일 유사 개체(쓰기 및 플러시를 구현해야 함) 또는 문자열 또는 os.파일 이름을 포함하는 PathLike 개체, 단순하게 저장할 위치 + 파일명이다
            - pickle_module (Any) : 메타데이터 및 객체를 피클링하는 데 사용되는 모듈
            - pickle_protocol (int) : 기본 프로토콜을 재정의하도록 지정할 수 있습니다.
            #
    - 일반적인 PyTorch 규칙은 .pt 파일 확장자를 사용하여 텐서를 저장하는 것입니다.

    - torch.save(object, path) 예제

In [ ]:
PATH = './weights/'

torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

2. torch.load
    - pickle 모듈을 이용하여 객체를 역직렬화하여 메모리에 할당합니다.

    - 형태 : torch.load(f, map_location=None, pickle_module=pickle, *, weights_only=False, **pickle_load_args)

    - torch.load(path) 이때 path: 불러올 위치 + 파일명


3. torch.nn.Module.load_state_dict: 역직렬화된 state_dict를 사용, 모델의 매개변수들을 불러옵니다. state_dict는 간단히 말해 각 체층을 매개변수 Tensor로 매핑한 Python 사전(dict) 객체입니다.

    - 형태 : torch.nn.Module.loadstatedict(dict)

    - state_dict를 이용하여, 모델 객체 내의 매개 변수 값을 초기화 합니다.

    - 예제


In [ ]:
model = torch.load(PATH + 'model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기

model.load_state_dict(checkpoint['model'])

optimizer.load_state_dict(checkpoint['optimizer'])

모델 가중치를 불러오기 위해서는, 먼저 동일한 모델의 인스턴스(instance)를 생성한 다음에 load_state_dict() 메소드를 사용하여 매개변수들을 불러옵니다.

추론(inference)을 하기 전에 model.eval() 메소드를 호출하여 드롭아웃(dropout)과 배치 정규화(batch normalization)를 평가 모드(evaluation mode)로 설정해야 합니다. 

그렇지 않으면 일관성 없는 추론 결과가 생성됩니다.

In [ ]:
model = models.vgg16() # 기본 가중치를 불러오지 않으므로 pretrained=True를 지정하지 않습니다.
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()